#Restic Ransomware Simulation & Auto‑Recovery Lab

Integrates automated Restic snapshots, simulates a ransomware attack, and then verifies and restores the latest clean baseline—all without manual intervention.

##Summary

This notebook spins up a self‑contained playground that

* sets up a Restic repo and captures an initial **baseline** snapshot
* launches a watchdog that backs up every file‑close (`--tag auto`)
  plus a background producer that keeps writing fresh log files
* runs a periodic scheduler that re‑tags the latest state as a new baseline
* simulates a ransomware attack by encrypting docs and tagging an **attack** snapshot
* automatically finds the latest *readable* baseline, restores it, verifies
  key files, and swaps the clean data back into place—deleting malicious
  artifacts and pruning old logs.

##Step 0: Persist everything to Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')          # Skip this cell if you don't need persistence

# Choose a root dir that survives VM resets if you mounted Drive
ROOT = "/content/drive/MyDrive/ransomware_lab"  # or "/content" if Drive not mounted
!mkdir -p $ROOT

Mounted at /content/drive


##Step 1: Install Tools

In [2]:
%%bash
sudo apt-get update -qq
sudo apt-get install -y restic gnupg tree

Reading package lists...
Building dependency tree...
Reading state information...
gnupg is already the newest version (2.2.27-3ubuntu2.4).
gnupg set to manually installed.
Suggested packages:
  sphinx-rtd-theme-common
The following NEW packages will be installed:
  restic tree
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 7,439 kB of archives.
After this operation, 20.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 restic amd64 0.12.1-2ubuntu0.3 [7,391 kB]
Fetched 7,439 kB in 1s (7,067 kB/s)
Selecting previously unselected package tree.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Selecting previously unselected package restic.
Preparing to unpack .../restic_0.12.1-2ubuntu0.3_amd64.deb ...
Unpacking 

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [3]:
#Install extra utilities
%%bash
sudo apt-get install -y inotify-tools   # for near real‑time backup
pip install --quiet watchdog            # Python file‑watcher library

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libinotifytools0
The following NEW packages will be installed:
  inotify-tools libinotifytools0
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 54.8 kB of archives.
After this operation, 233 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libinotifytools0 amd64 3.22.1.0-2 [22.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 inotify-tools amd64 3.22.1.0-2 [32.2 kB]
Fetched 54.8 kB in 0s (145 kB/s)
Selecting previously unselected package libinotifytools0:amd64.
(Reading database ... 126424 files and directories currently installed.)
Preparing to unpack .../libinotifytools0_3.22.1.0-2_amd64.deb ...
Unpacking libinotifytools0:amd64 (3.22.1.0-2) ...
Selecting previously unselected package inotify-tools.
Preparing to unpack .../inotify-tools_3.22.1.0-2_amd64.d

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


##Step 2: Configure Restic Repo

In [4]:
import os, subprocess, textwrap, getpass, json, pathlib, shutil, time

# Change these if you like
DATA_DIR      = f"{ROOT}/victim_data"
RESTIC_REPO   = f"{ROOT}/backup_repo"
RESTORE_DIR   = f"{ROOT}/restore"
os.makedirs(DATA_DIR,   exist_ok=True)
os.makedirs(RESTORE_DIR, exist_ok=True)

# One‑liner to set the repo password (DON'T lose it!)
os.environ["RESTIC_PASSWORD"] = "colab‑demo‑super‑secret"  # choose a strong one IRL

# Initialize repository once
if not pathlib.Path(RESTIC_REPO, "config").exists():
    !restic -r $RESTIC_REPO init

created restic repository 7554385b3c at /content/drive/MyDrive/ransomware_lab/backup_repo

Please note that knowledge of your password is required to access
the repository. Losing your password means that your data is
irrecoverably lost.


##Step 3: Generate sample "production" files

In [5]:
%%bash -s "$DATA_DIR"
TARGET=$1
mkdir -p "$TARGET/docs" "$TARGET/images"

# simple text data
echo "Quarterly revenue: \$123,456"  > "$TARGET/docs/report_Q1.txt"
echo "User list (PII redacted)"      > "$TARGET/docs/users.txt"

# simulate binary data
head -c 1M </dev/urandom > "$TARGET/images/raw_sensor_dump.bin"

tree -h "$TARGET"

[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  28]  report_Q1.txt
│   └── [  25]  users.txt
└── [4.0K]  images
    └── [1.0M]  raw_sensor_dump.bin

2 directories, 3 files


##Step 4: Define Helper to Print List of Snapshots

In [6]:
import subprocess
import sys
import os

def print_current_snapshots(restic_repo: str):
    """
    Prints the list of snapshots in the specified Restic repository.
    Relies on RESTIC_PASSWORD env var for authentication.
    """
    common_args = ["restic", "--repo", restic_repo]

    try:
        # Use --no-lock for faster listing if appropriate
        output = subprocess.check_output(
            common_args + ["snapshots", "--no-lock"],
            env=os.environ,
            text=True
        )
        print("🔍 Current snapshots in repository:")
        print(output)
    except subprocess.CalledProcessError as e:
        err = e.stderr if hasattr(e, 'stderr') and e.stderr else str(e)
        print("❗ Error listing snapshots:", err, file=sys.stderr)

##Step 5: Define Baseline Snapshot Helper

In [7]:
import subprocess, sys
from datetime import datetime

def ensure_baseline_snapshot(data_dir: str, restic_repo: str):
    """
    Checks for an existing Restic snapshot; if none, takes the baseline backup.
    """
    common_args = [
        "restic",
        "--repo", restic_repo
    ]

    # 1) List existing snapshots
    try:
        result = subprocess.run(
            common_args + ["snapshots", "--no-lock"],
            capture_output=True, text=True, check=True
        )
    except subprocess.CalledProcessError as e:
        print("❗ Error listing snapshots:", e.stderr, file=sys.stderr)
        sys.exit(1)

    # 2) If none exist, make the baseline
    if any(line.startswith("snapshot ") for line in result.stdout.splitlines()):
        print("✅  Baseline already exists; skipping.")
    else:
        #print("🚀  No baseline found—taking initial snapshot…")
        try:
            subprocess.run(
                common_args + ["backup", data_dir, "--tag", "baseline"],
                check=True
            )
            print(f"✅  Baseline complete at {datetime.now().isoformat()}")
        except subprocess.CalledProcessError as e:
            print("❗ Backup failed:", e.stderr, file=sys.stderr)
            sys.exit(1)

# Immediately take your golden snapshot tagged "baseline"
#ensure_baseline_snapshot(DATA_DIR, RESTIC_REPO)
# After setting up DATA_DIR, RESTIC_REPO, and exporting RESTIC_PASSWORD
#print_current_snapshots(RESTIC_REPO)

##Step 6: Start Watchdog that Snapshots on Every File-close

In [8]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import subprocess, os

class ResticOnClose(FileSystemEventHandler):
    def __init__(self, repo, path): self.repo, self.path = repo, path
    def on_closed(self, event):
        if event.is_directory: return
        print(f"[Watch] {event.src_path} closed → instant backup")
        subprocess.run(
            ["restic","-r",self.repo,"backup",self.path,"--tag","auto"],
            env=os.environ, check=False
        )

observer = Observer()
observer.schedule(ResticOnClose(RESTIC_REPO, DATA_DIR), str(DATA_DIR), recursive=True)
observer.start()
print("📡 Watchdog active — every write triggers a restic backup")

📡 Watchdog active — every write triggers a restic backup


##Step 7: Start Background Producer
Writes new log file every 15 seconds

In [9]:
import threading, random, string, pathlib, time
STREAM_DIR = pathlib.Path(DATA_DIR) / "logs"; STREAM_DIR.mkdir(exist_ok=True)

def producer(stop_event, interval=15):
    while not stop_event.is_set():
        fn = STREAM_DIR / f"log_{int(time.time())}.txt"
        fn.write_text(''.join(random.choices(string.ascii_letters, k=1024)))
        print(f"[Producer] wrote {fn.name}")
        time.sleep(interval)

stop_event = threading.Event()
prod_thr   = threading.Thread(target=producer, args=(stop_event,), daemon=True)
prod_thr.start()

##Step 8: Define Periodic Baseline Scheduler

In [10]:
import threading
import time

def start_periodic_baseline(data_dir: str, restic_repo: str, interval: int = 30):
    """
    Spawns a background thread that re-tags the latest state as a new baseline
    every `interval` seconds by calling ensure_baseline_snapshot().
    """
    def loop():
        while True:
            ensure_baseline_snapshot(data_dir, restic_repo)
            time.sleep(interval)

    t = threading.Thread(target=loop, daemon=True)
    t.start()
    print(f"🕒 Started periodic baseline thread (every {interval}s)")

start_periodic_baseline(DATA_DIR, RESTIC_REPO, interval=30)

🕒 Started periodic baseline thread (every 30s)


##Step 9: Define Helper to Restore Latest Baseline Snapshot that is Readable

In [11]:
import subprocess
import os
import pathlib
import shutil
import sys
import json
import tempfile
from datetime import datetime

def restore_latest_clean_baseline(restic_repo: str, restore_dir: str):
    """
    Attempts to restore the latest clean baseline snapshot.
    Iterates through snapshots tagged 'baseline' in reverse chronological order,
    restores to a temporary directory, verifies critical files, moves data,
    and returns the restored snapshot ID.
    """
    print("🔍 Searching for clean baseline snapshots...")

    # 1. List baseline snapshots
    try:
        proc = subprocess.run(
            ["restic", "-r", restic_repo, "snapshots", "--tag", "baseline", "--json"],
            capture_output=True, text=True, check=True, env=os.environ
        )
        snapshots = json.loads(proc.stdout)
    except subprocess.CalledProcessError as e:
        print("❌ Failed to list baseline snapshots:", e.stderr.strip(), file=sys.stderr)
        return False, None

    if not snapshots:
        print("❌ No baseline snapshots found.")
        return False, None

    # 2. Sort by timestamp descending
    for snap in snapshots:
        snap["parsed_time"] = datetime.fromisoformat(snap["time"].rstrip("Z"))
    snapshots.sort(key=lambda s: s["parsed_time"], reverse=True)

    restore_path = pathlib.Path(restore_dir)
    tmp_root = pathlib.Path(tempfile.mkdtemp(prefix="restic_tmp_"))

    try:
        # 3. Try each snapshot until valid
        for snap in snapshots:
            sid = snap["id"]
            print(f"🕵️ Trying snapshot {sid}...")

            tmp_dir = tmp_root / sid
            if tmp_dir.exists():
                shutil.rmtree(tmp_dir)
            tmp_dir.mkdir(parents=True)

            # Restore
            try:
                subprocess.run(
                    ["restic", "-r", restic_repo, "restore", sid, "--target", str(tmp_dir)],
                    capture_output=True, text=True, check=True
                )
            except subprocess.CalledProcessError as e:
                print(f"⚠️ Restore failed for {sid}: {e.stderr.strip()}")
                continue

            # Locate victim_data
            victim_dirs = list(tmp_dir.rglob("victim_data"))
            if not victim_dirs:
                print(f"❌ victim_data not found in {sid}")
                continue
            victim_path = victim_dirs[0]

            # Verify critical file readability
            critical = victim_path / "docs" / "report_Q1.txt"
            if not critical.exists():
                print(f"❌ report_Q1.txt missing in {sid}")
                continue
            try:
                with open(critical, 'r') as f:
                    f.read(1)
            except Exception as e:
                print(f"❌ Cannot read report_Q1.txt in {sid}: {e}")
                continue

            # 4. Prepare restore_dir
            if restore_path.exists():
                shutil.rmtree(restore_path)
            restore_path.mkdir(parents=True)

            # Move contents
            for item in tmp_dir.iterdir():
                shutil.move(str(item), str(restore_path / item.name))

            print(f"✅ Restored clean baseline {sid} into {restore_dir}")
            return True, sid

        print("❌ No clean baseline snapshot found.")
        return False, None

    finally:
        shutil.rmtree(tmp_root)

##Step 10: 30 Seconds Pause
Producer and Watchdog can create auto snapshots

In [13]:
import time, subprocess, os
print("⌛ Sleeping 30 s so real‑time activity accumulates …")
time.sleep(30)
print_current_snapshots(RESTIC_REPO)
#!restic -r $REPO_DIR snapshots | tail -n +3       # show newest snapshots

⌛ Sleeping 30 s so real‑time activity accumulates …
[Producer] wrote log_1753481765.txt
[Watch] /content/drive/MyDrive/ransomware_lab/victim_data/logs/log_1753481765.txt closed → instant backup
[Producer] wrote log_1753481780.txt
[Watch] /content/drive/MyDrive/ransomware_lab/victim_data/logs/log_1753481780.txt closed → instant backup
✅  Baseline complete at 2025-07-25T22:16:23.657343
🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
b37ff37f  2025-07-25 22:15:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
1425f92a  2025-07-25 22:15:22  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
9b6a6d5d  2025-07-25 22:15:35  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
19d9709e  2025-07-25 22:15:50  41d67b53b417  auto        /content/drive/MyDrive/ransomw

##Step 11: Simulate Ransomware attack

###Define Helper to Mark Simulated Ransomware Attack in Snapshots

In [14]:
import subprocess, os

def mark_attack_snapshot(data_dir: str, restic_repo: str):
    """
    Takes a Restic backup of data_dir and tags it 'attack' to mark
    the moment of the simulated ransomware event.
    """
    common_args = ["restic", "--repo", restic_repo]
    print("⚠️  Ransomware attack simulated — taking 'attack' snapshot…")
    subprocess.run(
        common_args + ["backup", data_dir, "--tag", "attack"],
        check=True, env=os.environ
    )
    print("✅  'attack' snapshot complete.")

###Encrypt the files and Simulate Attack

In [15]:
%%bash -s "$DATA_DIR"
VICTIM=$1
# Encrypt *.txt as a stand‑in for ransomware, then delete originals
cd "$VICTIM/docs"
for f in *.txt; do
  openssl enc -aes-256-cbc -md sha256 -salt -in "$f" -out "${f}.enc" -pass pass:evil123
  shred -u "$f"
done
echo "After attack:"
tree -h "$VICTIM"

[Watch] /content/drive/MyDrive/ransomware_lab/victim_data/docs/report_Q1.txt.enc closed → instant backup
After attack:
[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  48]  report_Q1.txt.enc
│   └── [  48]  users.txt.enc
├── [4.0K]  images
│   └── [1.0M]  raw_sensor_dump.bin
└── [4.0K]  logs
    ├── [1.0K]  log_1753481720.txt
    ├── [1.0K]  log_1753481735.txt
    ├── [1.0K]  log_1753481750.txt
    ├── [1.0K]  log_1753481765.txt
    ├── [1.0K]  log_1753481780.txt
    └── [1.0K]  log_1753481795.txt

3 directories, 9 files


*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.
*** WARNING : deprecated key derivation used.
Using -iter or -pbkdf2 would be better.


In [16]:
##mark the attack in Restic
mark_attack_snapshot(DATA_DIR, RESTIC_REPO)

⚠️  Ransomware attack simulated — taking 'attack' snapshot…
✅  'attack' snapshot complete.


In [17]:
print_current_snapshots(RESTIC_REPO)

🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
b37ff37f  2025-07-25 22:15:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
1425f92a  2025-07-25 22:15:22  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
9b6a6d5d  2025-07-25 22:15:35  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
19d9709e  2025-07-25 22:15:50  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
acf00fc9  2025-07-25 22:15:52  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
8e403c4b  2025-07-25 22:16:05  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
5b341f06  2025-07-25 22:16:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
d63a9614  2025-07-25 22:16:23  41d67b53b417  b

##Step 12: Attempt to Read
Will fail

In [18]:
import pathlib, sys

plaintext = pathlib.Path(DATA_DIR) / "docs" / "report_Q1.txt"   # original file path

try:
    text = plaintext.read_text()          # should raise FileNotFoundError
    print("❌  UNEXPECTED: plaintext still readable!", text[:100])
    sys.exit(1)
except (FileNotFoundError, PermissionError):
    print("✅  Expected failure: plaintext is gone or unreadable — ransomware succeeded.")

✅  Expected failure: plaintext is gone or unreadable — ransomware succeeded.


##Step 13: Restore the Latest Clean Snapshot

In [19]:
#List all snapshots
print_current_snapshots(RESTIC_REPO)

🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
b37ff37f  2025-07-25 22:15:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
1425f92a  2025-07-25 22:15:22  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
9b6a6d5d  2025-07-25 22:15:35  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
19d9709e  2025-07-25 22:15:50  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
acf00fc9  2025-07-25 22:15:52  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
8e403c4b  2025-07-25 22:16:05  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
5b341f06  2025-07-25 22:16:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
d63a9614  2025-07-25 22:16:23  41d67b53b417  b

In [20]:
success, restored_id = restore_latest_clean_baseline(RESTIC_REPO, RESTORE_DIR)

🔍 Searching for clean baseline snapshots...
🕵️ Trying snapshot a3b93b4ed7c6cbbb7b18a06bad97756fea1255d2abae4e029c8e973b0127fb1f...
❌ report_Q1.txt missing in a3b93b4ed7c6cbbb7b18a06bad97756fea1255d2abae4e029c8e973b0127fb1f
🕵️ Trying snapshot d63a96143866b6f4f165dc900e1d62a1229c82e16b9e28e79fc36514669a2b63...
✅ Restored clean baseline d63a96143866b6f4f165dc900e1d62a1229c82e16b9e28e79fc36514669a2b63 into /content/drive/MyDrive/ransomware_lab/restore


##Step 14: Verify Restored File Exists and Matches Expected Content

In [21]:
%%bash
# 1. Point to your restore directory
export RESTORE_DIR="/content/drive/MyDrive/ransomware_lab/restore"

# 2. Locate the nested victim_data folder
VICTIM_PATH=$(find "$RESTORE_DIR" -type d -name victim_data | head -n1)
if [ -z "$VICTIM_PATH" ]; then
  echo "❌ Could not find victim_data under $RESTORE_DIR"
  exit 1
fi

echo "✅ Restored data is at: $VICTIM_PATH"

# 3. List the docs you recovered
echo; echo "Recovered docs:"
tree -h "$VICTIM_PATH/docs" | sed -n '1,5p'

# 4. Show the contents of report_Q1.txt
echo; echo "Contents of report_Q1.txt:"
cat "$VICTIM_PATH/docs/report_Q1.txt"

✅ Restored data is at: /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data

Recovered docs:
[4.0K]  /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data/docs
├── [  28]  report_Q1.txt
└── [  25]  users.txt

0 directories, 2 files

Contents of report_Q1.txt:
Quarterly revenue: $123,456


In [22]:
import pathlib, hashlib, sys

# Adjust this to your actual restore path variable if needed
restore_root = pathlib.Path(RESTORE_DIR)
victim = next(restore_root.rglob("victim_data"), None)
if victim is None:
    print("❌  victim_data not found!"); sys.exit(1)

file = victim / "docs" / "report_Q1.txt"
if not file.exists():
    print("❌  report_Q1.txt missing!"); sys.exit(1)

content = file.read_text()
assert "Quarterly revenue: $123,456" in content, "Content mismatch!"

sha = hashlib.sha256(file.read_bytes()).hexdigest()
print(f"✅  report_Q1.txt verified — SHA‑256: {sha[:12]}…")

✅  Baseline complete at 2025-07-25T22:17:24.894946
✅  report_Q1.txt verified — SHA‑256: 2758452adf6e…


##Step 15: Compare Original to Restored
Sanity Check

In [ ]:
%%bash -s "$DATA_DIR" "$RESTORE_DIR"
# $1 is the live data dir, $2 is where Restic put your restore tree
ORIG="$1"
RESTORE_ROOT="$2"

# 2. Locate the restored victim_data under RESTORE_ROOT
RESTORED_V=$(find "$RESTORE_ROOT" -type d -name victim_data | head -n1)
if [ -z "$RESTORED_V" ]; then
  echo "❌  Could not find victim_data under $RESTORE_ROOT"
  exit 1
fi

echo "✅  Restored data is at: $RESTORED_V"
echo

# 3. Compare original vs restored
echo "Comparing live vs restored:"
echo "  live:     $ORIG"
echo "  restored: $RESTORED_V"
echo

diff -rq "$ORIG" "$RESTORED_V" \
  && echo "✅  No unexpected differences." \
  || echo "🔶  Differences above are expected for encrypted docs."

##Step 16: Stop Background Producer and Watchdog

In [23]:
stop_event.set()               # stop producer loop
observer.stop(); observer.join()
print("✅ Real‑time components shut down")

✅ Real‑time components shut down


In [31]:
print_current_snapshots(RESTIC_REPO)

🔍 Current snapshots in repository:
ID        Time                 Host          Tags        Paths
----------------------------------------------------------------------------------------------------------
b37ff37f  2025-07-25 22:15:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
1425f92a  2025-07-25 22:15:22  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
9b6a6d5d  2025-07-25 22:15:35  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
19d9709e  2025-07-25 22:15:50  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
acf00fc9  2025-07-25 22:15:52  41d67b53b417  baseline    /content/drive/MyDrive/ransomware_lab/victim_data
8e403c4b  2025-07-25 22:16:05  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
5b341f06  2025-07-25 22:16:20  41d67b53b417  auto        /content/drive/MyDrive/ransomware_lab/victim_data
d63a9614  2025-07-25 22:16:23  41d67b53b417  b

##FIX THIS: Optional: Prune Workspace
* Locating the deepest nested 'victim_data' directory inside restore_root
* Removing any '.enc' encrypted files
* Moving the restored data to the original victim_data_path
* Pruning the logs directory to keep only the latest .txt file
* Deleting all contents of the restore_root directory

In [32]:
import os
import shutil
import pathlib

def finalize_restoration(restore_root: str, victim_data_path: str):
    restore_root = pathlib.Path(restore_root)
    victim_data_path = pathlib.Path(victim_data_path)

    # Locate the deepest nested victim_data directory
    nested_victim_dirs = sorted(restore_root.rglob("victim_data"), key=lambda p: len(str(p)), reverse=True)
    if not nested_victim_dirs:
        print("❌ No victim_data directory found in restore root.")
        return False

    restored_victim = nested_victim_dirs[0]
    print(f"✅ Found restored victim_data at: {restored_victim}")

    # Remove encrypted files
    for enc_file in restored_victim.rglob("*.enc"):
        try:
            enc_file.unlink()
            print(f"🗑️ Deleted encrypted file: {enc_file}")
        except Exception as e:
            print(f"⚠️ Failed to delete {enc_file}: {e}")

    # Replace original victim_data with restored version
    if victim_data_path.exists():
        shutil.rmtree(victim_data_path)
    shutil.copytree(restored_victim, victim_data_path)
    print(f"📦 Moved restored data to: {victim_data_path}")

    # Prune logs to keep only the latest .txt file
    logs_dir = victim_data_path / "logs"
    if logs_dir.exists():
        txt_files = sorted(logs_dir.glob("*.txt"), key=lambda f: f.stat().st_mtime, reverse=True)
        if txt_files:
            latest = txt_files[0]
            for f in txt_files[1:]:
                try:
                    f.unlink()
                    print(f"🗑️ Deleted old log file: {f.name}")
                except Exception as e:
                    print(f"⚠️ Failed to delete log file {f.name}: {e}")
            print(f"📝 Kept latest log file: {latest.name}")
        else:
            print("ℹ️ No .txt log files found to prune.")
    else:
        print("ℹ️ No logs directory found.")

    # Delete all contents of the restore_root directory
    for item in restore_root.iterdir():
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink()
            print(f"🧹 Deleted: {item}")
        except Exception as e:
            print(f"⚠️ Failed to delete {item}: {e}")

    return True

In [33]:
if success:
  finalize_restoration(
      restore_root=RESTORE_DIR,
      victim_data_path=DATA_DIR
  )

✅ Found restored victim_data at: /content/drive/MyDrive/ransomware_lab/restore/content/drive/MyDrive/ransomware_lab/victim_data
📦 Moved restored data to: /content/drive/MyDrive/ransomware_lab/victim_data
🗑️ Deleted old log file: log_1753481765.txt
🗑️ Deleted old log file: log_1753481750.txt
🗑️ Deleted old log file: log_1753481735.txt
🗑️ Deleted old log file: log_1753481720.txt
📝 Kept latest log file: log_1753481780.txt
🧹 Deleted: /content/drive/MyDrive/ransomware_lab/restore/content


In [34]:
%%bash
#!/bin/bash

# Define the root directory
VICTIM_PATH="/content/drive/MyDrive/ransomware_lab/victim_data"
RESTORE_PATH="/content/drive/MyDrive/ransomware_lab/restore"

# Check if the directory exists
if [ ! -d "$VICTIM_PATH" ]; then
  echo "❌ Directory not found: $VICTIM_PATH"
  exit 1
fi
if [ ! -d "$RESTORE_PATH" ]; then
  echo "❌ Directory not found: $RESTORE_PATH"
  exit 1
fi

# Print the tree with human-readable sizes
echo "📁 Directory tree for: $VICTIM_PATH"
echo

tree -h "$VICTIM_PATH"
# Print the tree with human-readable sizes
echo "📁 Directory tree for: $RESTORE_PATH"
echo

tree -h "$RESTORE_PATH"

📁 Directory tree for: /content/drive/MyDrive/ransomware_lab/victim_data

[4.0K]  /content/drive/MyDrive/ransomware_lab/victim_data
├── [4.0K]  docs
│   ├── [  28]  report_Q1.txt
│   └── [  25]  users.txt
├── [4.0K]  images
│   └── [1.0M]  raw_sensor_dump.bin
└── [4.0K]  logs
    └── [1.0K]  log_1753481780.txt

3 directories, 4 files
📁 Directory tree for: /content/drive/MyDrive/ransomware_lab/restore

[4.0K]  /content/drive/MyDrive/ransomware_lab/restore

0 directories, 0 files
